In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline
from SparseFactorization.sparse_factorization import *
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np

/dfs/scratch0/maxlam/env3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Define roots of unity
def w_n(n):
    return np.e**((2 * np.pi * 1j) / n)

In [3]:
# Omegas
def Omega_n(n):
    values = [w_n(i)**(-i) if i >= 1 else 1 for i in range(n)]
    return np.diag(values)

Omega_n(4)

array([[1.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 1.+2.44929360e-16j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 1.+2.44929360e-16j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        1.+6.10622664e-16j]])

In [4]:
# Define B_2 and B_4
def B_n(n):
    
    # Identities
    ident = np.eye(n//2)
    stacked_idnt = np.concatenate([ident, ident], axis=0)
    
    # Omegas
    o1 = Omega_n(n//2)
    o2 = -Omega_n(n//2)
    stacked_omeg = np.concatenate([o1, o2], axis=0)
    
    return np.concatenate([stacked_idnt, stacked_omeg], axis=1)

B_n(4)

array([[ 1.+0.0000000e+00j,  0.+0.0000000e+00j,  1.+0.0000000e+00j,
         0.+0.0000000e+00j],
       [ 0.+0.0000000e+00j,  1.+0.0000000e+00j,  0.+0.0000000e+00j,
         1.+2.4492936e-16j],
       [ 1.+0.0000000e+00j,  0.+0.0000000e+00j, -1.-0.0000000e+00j,
        -0.-0.0000000e+00j],
       [ 0.+0.0000000e+00j,  1.+0.0000000e+00j, -0.-0.0000000e+00j,
        -1.-2.4492936e-16j]])

In [5]:
# Define F_8 and F_4
def F_8():
    # First b8 factor
    B_8 = B_n(8)
    
    # Stack b4s
    B_4 = B_n(4)
    rows = []    
    for i in range(2):
        row = [np.zeros(B_4.shape) for j in range(i)] + [B_4] + [np.zeros(B_4.shape) for j in range(2-i-1)]
        rows.append(np.concatenate(row, axis=1))
    B_4_factor = np.concatenate(rows, axis=0)
    
    # Stack b2s
    B_2 = B_n(2)
    rows = []
    for i in range(4):
        row = [np.zeros(B_2.shape) for j in range(i)] + [B_2] + [np.zeros(B_2.shape) for j in range(4-i-1)]
        rows.append(np.concatenate(row, axis=1))
    B_2_factor = np.concatenate(rows, axis=0)
    
    # Permutation matrix
    perm = np.array([
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0 ,0 ,0 ,0 ,0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1]
    ])
    
    return B_8.dot(B_4_factor).dot(B_2_factor).dot(perm), [B_8, B_4_factor, B_2_factor, perm]
    
F_8()

(array([[ 1.+0.00000000e+00j,  1.+0.00000000e+00j,  1.+0.00000000e+00j,
          1.+0.00000000e+00j,  1.+0.00000000e+00j,  1.+0.00000000e+00j,
          1.+0.00000000e+00j,  1.+0.00000000e+00j],
        [ 1.+0.00000000e+00j,  1.+2.44929360e-16j,  1.+2.44929360e-16j,
          1.+4.89858720e-16j, -1.+0.00000000e+00j, -1.-2.44929360e-16j,
         -1.-2.44929360e-16j, -1.-4.89858720e-16j],
        [ 1.+0.00000000e+00j,  1.+2.44929360e-16j, -1.+0.00000000e+00j,
         -1.-2.44929360e-16j,  1.+0.00000000e+00j,  1.+2.44929360e-16j,
         -1.+0.00000000e+00j, -1.-2.44929360e-16j],
        [ 1.+0.00000000e+00j,  1.+6.10622664e-16j, -1.-2.44929360e-16j,
         -1.-8.55552023e-16j, -1.+0.00000000e+00j, -1.-6.10622664e-16j,
          1.+2.44929360e-16j,  1.+8.55552023e-16j],
        [ 1.+0.00000000e+00j, -1.+0.00000000e+00j,  1.+0.00000000e+00j,
         -1.+0.00000000e+00j,  1.+0.00000000e+00j, -1.+0.00000000e+00j,
          1.+0.00000000e+00j, -1.+0.00000000e+00j],
        [ 1.+0.00000

In [6]:
# Helper to convert complex matrices into pytorch form (n,m,...,2)
def to_pytorch(complex_matrix):
    return np.stack([complex_matrix.real, complex_matrix.imag], axis=len(complex_matrix.shape))

# Helper to convert complex matrices in pytorch form into numpy
def to_numpy(complex_matrix):
    newshp = complex_matrix.shape[:-1]
    return complex_matrix[:,:,0].reshape(newshp) + complex_matrix[:,:,-1].reshape(newshp)*1j

In [76]:
###########################################################################
# This experiment factorized F8 using fine grained sparsity constraints.  
# Given the true factors of F_8, we force all nnzs in the corresponding optimized       
# matrix to be 0 if it is also a 0 in the true factor
#
# E.g: if
# F_4_1 = [[1, 0, 1, 0],
#          [0, 1, 0, w4],
#          [1, 0, -1, 0],
#          [0, 1, 0, -w4]
# where F_4 = F_4_1 * F_4_2 * F_4_perm
# Then we force factor 1 the optimize matrix to have form (enforcing sparsity constraint)
# 
# [[a, 0, b, 0],
#  [0, c, 0, d],
#  [e, 0, f, 0],
#  [0, g, 0, h]
#
# Since the 0s all match, I call this the fine-grained sparsity constraint
    

%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import SparseFactorization
from SparseFactorization.sparse_factorization import *

F_8_matrix, F_8_true_factors = F_8()

# Sparsity enforcements
B_8_sparsity = (F_8_true_factors[0] != 0) * 1
B_4_sparsity = (F_8_true_factors[1] != 0) * 1
B_2_sparsity = (F_8_true_factors[2] != 0) * 1

# Permutation enforce
perm_values = F_8_true_factors[-1]

# Dbg
print(B_8_sparsity)
print(B_4_sparsity)
print(B_2_sparsity)
print(perm_values)

# Constraints to pytorch form
B_8_sparsity = to_pytorch(B_8_sparsity)
B_4_sparsity = to_pytorch(B_4_sparsity)
B_2_sparsity = to_pytorch(B_2_sparsity)
perm_values = to_pytorch(perm_values)

# Factorize!
hyperparameters = {
    "learning_rate" : 1e-2,
    "l1_parameter" : 0, 
    "pruning_threshold" : 0, 
    "training_iters" : 8000,
    "log_every" : 1000,
    
    # Here we 4 factors (XYZP)
    "matrix_initializations" : {
        0: np.stack([np.eye(8), np.eye(8)], axis=2),
        1: np.stack([np.eye(8), np.eye(8)], axis=2),
        2: np.stack([np.eye(8), np.eye(8)], axis=2),        
        3: np.stack([np.eye(8), np.eye(8)], axis=2),
    },
    
    # Here we apply sparsity constraints according to the sparsity of the butterfly patterns
    "sparsity_constraints" : {
        0:B_8_sparsity,
        1:B_4_sparsity,
        2:B_2_sparsity,
    },
    
    # Here we apply value constraints to the permutation matrix. P=permutation matrix
    "value_constraints" : {
        3: perm_values
    },
    
    "is_complex": True
}

factorizer = SparseFactorizationWithEnforcedStructurePytorch(hyperparameters=hyperparameters)
recovered_factors, details = factorizer.factorize(to_pytorch(F_8_matrix))
recovered_factors = [to_numpy(x) for x in recovered_factors]

# Visualize each of the recovered_factors and the actual factors
plt.figure(1, figsize=(15,15))

actual_factors = F_8_true_factors

# Put actual and recovered factors side by side
np.set_printoptions(precision=3)

print("==================")
print("Compare real parts")
print("==================")

print("Actual 0\n", actual_factors[0].real)
print("Recovered 0\n", recovered_factors[0].real)

print("Actual 1\n", actual_factors[1].real)
print("Recovered 1\n", recovered_factors[1].real)

print("Actual 2\n", actual_factors[2].real)
print("Recovered 2\n", recovered_factors[2].real)

print("Actual 3\n", actual_factors[3].real)
print("Recovered 3\n", recovered_factors[3].real)

print("==================")
print("Compare imag parts")
print("==================")

print("Actual 0\n", actual_factors[0].imag)
print("Recovered 0\n", recovered_factors[0].imag)

print("Actual 1\n", actual_factors[1].imag)
print("Recovered 1\n", recovered_factors[1].imag)

print("Actual 2\n", actual_factors[2].imag)
print("Recovered 2\n", recovered_factors[2].imag)

print("Actual 3\n", actual_factors[3].imag)
print("Recovered 3\n", recovered_factors[3].imag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[1 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 1]]
[[1 0 1 0 0 0 0 0]
 [0 1 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0]
 [0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 0 1]]
[[1 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1]]
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]]
SparseFactorizationWithL1AndPruningPytorch: Frob error: 13.8564, Loss: 13.8564, Variables NNzs: [16, 16, 16, 8], Sum NNzs: 56
SparseFactorizationWithL1AndPruningPytorch: Frob error: 0.0581686, Loss: 0.0581686, Variables NNzs: [16, 16, 16, 8], Sum NNzs: 56
SparseFactorizationWithL1AndPruningPytorch: 

<Figure size 1080x1080 with 0 Axes>

In [11]:
###########################################################################
# This experiment factorizes F8 using coarse grained sparsity constraints.  
#
# Given F_8 =
# B_8 [[B4 0],[0, B4]] [[B2, 0, 0, 0], [0, B2, 0, 0], [0, 0, B2, 0], [0, 0, 0, B2]] Perm
#
# In this experiment we enforce the second and third factor to have 0s as in the form form above.
# Since the 0s of the Is in the Bs are not captured these are called coarse grained sparsity constraints.

%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import SparseFactorization
from SparseFactorization.sparse_factorization import *

F_8_matrix, F_8_true_factors = F_8()

# Sparsity enforcements
#B_8_sparsity = (F_8_true_factors[0] != 0) * 1
#B_4_sparsity = (F_8_true_factors[1] != 0) * 1
#B_2_sparsity = (F_8_true_factors[2] != 0) * 1

# B_8 has no sparsity constraints
# B_4 looks like: [B_4 0], [0, B4]
B_4_sparsity = np.array([
    [1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1],    
])

B_2_sparsity = np.array([
    [1, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 1],    
])

# Permutation enforce
perm_values = F_8_true_factors[-1]

# Dbg
#print(B_8_sparsity)
print(B_4_sparsity)
print(B_2_sparsity)
print(perm_values)

# Constraints to pytorch form
#B_8_sparsity = to_pytorch(B_8_sparsity)
B_4_sparsity = to_pytorch(B_4_sparsity)
B_2_sparsity = to_pytorch(B_2_sparsity)
perm_values = to_pytorch(perm_values)

# Factorize!
hyperparameters = {
    "learning_rate" : 8e-3,
    "l1_parameter" : .0001, 
    "pruning_threshold" : 1e-4, 
    "training_iters" : 8000,
    "log_every" : 1000,
    
    # Here we 4 factors (XYZP)
    "matrix_initializations" : {
        0: np.stack([np.eye(8), np.eye(8)], axis=2),
        1: np.stack([np.eye(8), np.eye(8)], axis=2),
        2: np.stack([np.eye(8), np.eye(8)], axis=2),        
        3: np.stack([np.eye(8), np.eye(8)], axis=2),
    },
    
    # Here we apply sparsity constraints according to the sparsity of the butterfly patterns
    "sparsity_constraints" : {
        #0:B_8_sparsity,
        1:B_4_sparsity,
        2:B_2_sparsity,
    },
    
    # Here we apply value constraints to the permutation matrix. P=permutation matrix
    "value_constraints" : {
        3: perm_values
    },
    
    "is_complex": True
}

factorizer = SparseFactorizationWithEnforcedStructurePytorch(hyperparameters=hyperparameters)
recovered_factors, details = factorizer.factorize(to_pytorch(F_8_matrix))
recovered_factors = [to_numpy(x) for x in recovered_factors]

# Visualize each of the recovered_factors and the actual factors
plt.figure(1, figsize=(15,15))

actual_factors = F_8_true_factors

# Put actual and recovered factors side by side
np.set_printoptions(precision=3)

print("==================")
print("Compare real parts")
print("==================")

print("Actual 0\n", actual_factors[0].real)
print("Recovered 0\n", recovered_factors[0].real)

print("Actual 1\n", actual_factors[1].real)
print("Recovered 1\n", recovered_factors[1].real)

print("Actual 2\n", actual_factors[2].real)
print("Recovered 2\n", recovered_factors[2].real)

print("Actual 3\n", actual_factors[3].real)
print("Recovered 3\n", recovered_factors[3].real)

print("==================")
print("Compare imag parts")
print("==================")

print("Actual 0\n", actual_factors[0].imag)
print("Recovered 0\n", recovered_factors[0].imag)

print("Actual 1\n", actual_factors[1].imag)
print("Recovered 1\n", recovered_factors[1].imag)

print("Actual 2\n", actual_factors[2].imag)
print("Recovered 2\n", recovered_factors[2].imag)

print("Actual 3\n", actual_factors[3].imag)
print("Recovered 3\n", recovered_factors[3].imag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[1 1 1 1 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 1 1]]
[[1 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1]]
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]]
SparseFactorizationWithL1AndPruningPytorch: Frob error: 13.8564, Loss: 13.8628, Variables NNzs: [128, 32, 16, 8], Sum NNzs: 184
SparseFactorizationWithL1AndPruningPytorch: Frob error: 4.74024, Loss: 4.74699, Variables NNzs: [100, 32, 16, 8], Sum NNzs: 156
SparseFactorizationWithL1AndPruningPytorch: Frob error: 0.0587462, Loss: 0.0654232, Variables NNzs: [64, 28, 12, 8], Sum NNzs: 112
SparseFactorizationWithL1AndPruningPytorch: Frob error: 0.0681159,

<Figure size 1080x1080 with 0 Axes>